In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown
import vaccines as lib
import sys
import locale
sign = lambda x: 1 if x >= 0 else -1

# Dados e configurações
plt.rcParams["figure.figsize"] = (10,6.66)
pd.set_option('display.max_rows', None)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

data = pd.read_pickle('data.pkl.gz')
hospitalization = pd.read_pickle('hospitalization.pkl.gz')

In [2]:
display(Markdown("## Ferramenta para determinação do impacto da vacinação nas internações por COVID-19"))
default_plan = """95%"""
region_options = [(v, k) for k, v in data['name'].to_dict().items()]
vaccine_options = [('Coronavac/sinovac/butantan', (21, 20, 0.95)), ('Astrazeneca/Fiocruz', (90, 22, 0.95))]


@widgets.interact(
    region=widgets.Dropdown(options=region_options, description="UF"),
    vaccines=widgets.IntText(500_000, description="Doses"),
    rate=widgets.IntText(6_000, description="dose/dia"),
    vaccine=widgets.Dropdown(options=vaccine_options, description="Tipo vacina"),
    plan=widgets.Textarea(default_plan, description="Metas"),
    coarse=widgets.Checkbox(False, description='Agrupar de 10 em 10 anos'),
)
def run(vaccines, region, rate, vaccine, plan, coarse):
    global events, age_distribution, severe, result
    
    second_dose_delay, immunization_delay, eff = vaccine
    rates = lib.compute_schedule(rate, vaccines)
    age_distribution = lib.population_80_plus(data.loc[region, "age_distribution"], coarse=coarse)
    
    plan = lib.SimpleDosesRatePlan.from_source(plan, age_distribution, rate=rate, delay=second_dose_delay, max_doses=vaccines // 2)
    plan.execute()
    events = plan.summary()
    
    severe = hospitalization.loc[region].iloc[::-1]
    if coarse:
        severe = lib.coarse_distribution(severe)
        
    duration = lib.by_periods(events['day'].max() + immunization_delay + second_dose_delay, 30)
    result = lib.VaccinationCampaign(events, duration=duration)
    
    pressure = result.damage_curve(severe, delay=immunization_delay, efficiency=eff)
    deaths = result.damage_curve(severe, delay=immunization_delay)
    display(Markdown(f"""
## Resultados
* **Total de doses:** {int(result.vaccines):n}
* **Pessoas vacinadas:** {int(result.vaccines // 2):n}
* **Dias de vacinação:** {result.campaign_duration}
* **Redução na hospitalização:** {100 - 100 * pressure.iloc[-1]:.1f}%
* **Redução dos óbitos:** {100 - 100 * deaths.iloc[-1]:.1f}%
    """))
    result.plot_hospitalization_pressure_curve(pressure, as_pressure=True)
    result.plot_vaccination_schedule()
    plt.show()
    
    display(Markdown(f"""
## Observações

O primeiro gráfico mostra a diminuição na pressão hospitalar esperada 
devido à proteção conferida pelas vacinas. A simulação considera que a 
taxa de infecção se mantêm constante, o que é uma suposição conservadora, 
especialmente em níveis mais altos de vacinação.
    """))


## Ferramenta para determinação do impacto da vacinação nas internações por COVID-19

interactive(children=(IntText(value=500000, description='Doses'), Dropdown(description='UF', options=(('Acre',…

In [3]:
for m in list(sys.modules):
    if m.startswith('vaccines'):
        del sys.modules[m]
import vaccines as lib

# plan = lib.SimpleDosesRatePlan.from_source('90%', age_distribution, rate=4000, delay=30)
# plan.execute()
# plan.summary()

In [4]:
events

,day,age,doses,phase,acc,fraction
0,0,80,6000,1,6000,0.621697
1,1,80,3168,1,9168,0.949953
2,2,75,6000,1,6000,0.677048
3,3,75,2418,1,8418,0.949898
4,4,70,6000,1,6000,0.462356
5,5,70,6000,1,12000,0.924713
6,6,70,328,1,12328,0.949988
7,7,65,6000,1,6000,0.325981
8,8,65,6000,1,12000,0.651961
9,9,65,5485,1,17485,0.949962
